In [ ]:
import numpy as np  # type: ignore
import matplotlib.pyplot as plt  # type: ignore
import numpy.linalg as lin  # type: ignore
from mpl_toolkits.mplot3d import Axes3D  # type: ignore
import plotly.graph_objects as go  # type: ignore
from scipy.signal import spectrogram  # type: ignore
from scipy.stats import gaussian_kde
from scipy.ndimage import gaussian_filter
from scipy.optimize import minimize


# Définition des constantes
rho_0 = 917  # kg/m^3, masse volumique de référence de la glace
beta = 5.4 * 10**(-5)  # K^-1, coefficient de dilatation thermique de la glace
rho_air = 1.341  # kg/m^3, masse volumique de l'air à -10 °C
vair = 60  # m/s, la vitesse de l'air
Tair = 263.15  # K, température de l'air
rho = rho_0 * (1 - beta * (Tair - 273.15))  # la masse volumique du débris de glace
omega = 347  # rad/s
g = 9.81  # m/s^2

# Fonction de calcul de la masse
def m(R):
    return rho * 4 / 3 * np.pi * R**3  # la masse du débris de glace

# Fonction pour le système différentiel
def f(v, R, Cd):
    Vair = np.array([vair, 0, 0])
    return -(1 / 2) * rho_air * Cd * np.pi * (R**2) * lin.norm(v - Vair) * (v - Vair) / m(R)

# Méthode de Runge-Kutta d'ordre 4
def RK4(f, v0, dt, T_f, R, Cd):
    tab_v = [v0]
    v = v0
    t = 0
    tab_t = [t]
    while t < T_f:
        k1 = f(v, R, Cd)
        k2 = f(v + (dt / 2) * k1, R, Cd)
        k3 = f(v + (dt / 2) * k2, R, Cd)
        k4 = f(v + dt * k3, R, Cd)
        v = v + (dt / 6) * (k1 + 2 * k2 + 2 * k3 + k4)
        t = t + dt
        tab_v.append(v)
        tab_t.append(t)
    return np.array(tab_v), np.array(tab_t)

# Méthode pour calculer les positions
def calculate_positions(v_vectors, t, initial_positions):
    n = len(v_vectors)
    dt = t[1] - t[0]
    positions = np.zeros((n, 3))
    positions[0] = initial_positions
    for i in range(1, n):
        for j in range(3):
            positions[i, j] = positions[i - 1, j] + (v_vectors[i, j] + v_vectors[i - 1, j]) / 2 * dt
    return positions

# Simulation de 1000 trajectoires
n_simulations = 900
trajectories = []
energies = []
times = []

# Paramètres de la simulation
T_f = 0.5  # Temps final (s)
dt = 1/(2**10)  # Pas de temps (s)

for _ in range(n_simulations):
    # Génération des conditions initiales aléatoires
    x0 = np.random.uniform(0, 0.1)
    theta = np.random.uniform(0, 2 * np.pi)
    y0 = 0.5 * np.sqrt(x0) * np.cos(theta)
    z0 = 0.5 * np.sqrt(x0) * np.sin(theta)
    v0 = np.array([0, -z0 * omega, y0 * omega])

    # Génération des paramètres aléatoires
    Cd = max(np.random.normal(0.47, 0.05), 0)  # Cd doit être positif
    R = max(np.random.normal(0.005, 0.001), 0)  # R doit être positif

    # Résolution de l'équation différentielle
    v_vectors, t = RK4(f, v0, dt, T_f, R, Cd)
    positions = calculate_positions(v_vectors, t, (x0, 0, 0))

    # Stockage des trajectoires
    trajectories.append(positions)

    # Calcul de l'énergie cinétique finale
    energy = 0.5 * m(R) * lin.norm(v_vectors[-1])**2
    energies.append(energy)
    times.append(t[-1])

# Visualisation des trajectoires
# Visualisation interactive avec Plotly
fig = go.Figure()



for traj in trajectories:
    fig.add_trace(go.Scatter3d(
        x=traj[:, 0],
        y=traj[:, 1],
        z=traj[:, 2],
        mode='lines',
        line=dict(color='black', width=2),
        opacity=0.5
    ))

fig.update_layout(
    title="Trajectoires des débris de glace (Simulation interactive)",
    scene=dict(
        xaxis_title="x (m)",
        yaxis_title="y (m)",
        zaxis_title="z (m)"
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)
# Génération des points pour représenter le volume
n_points_x = 50  # Résolution pour x
n_points_theta = 100  # Résolution pour theta

x_vals = np.linspace(0, 0.5, n_points_x)
theta_vals = np.linspace(0, 2 * np.pi, n_points_theta)

X, Theta = np.meshgrid(x_vals, theta_vals)
Y = 0.5 * np.sqrt(X) * np.cos(Theta)
Z = 0.5 * np.sqrt(X) * np.sin(Theta)

# Ajout du volume à la visualisation
fig.add_trace(go.Surface(
    x=X,
    y=Y,
    z=Z,
    opacity=0.5,
    colorscale='Viridis',
    showscale=False
))

# Mise à jour des paramètres de la visualisation
fig.update_layout(
    title="Trajectoires des débris de glace et volume",
    scene=dict(
        xaxis_title="x (m)",
        yaxis_title="y (m)",
        zaxis_title="z (m)"
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

fig.update_layout(
    title="Trajectoires des débris de glace et volume",
    scene=dict(
        xaxis=dict(title="x (m)", range=[0, 0.5]),
        yaxis=dict(title="y (m)", range=[-1, 1]),
        zaxis=dict(title="z (m)", range=[-1, 1])
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)



fig.show()

# Analyse du spectre d'énergie
plt.figure(figsize=(8, 6))
plt.hist(energies, bins=30, density=True, alpha=0.7, color='b', edgecolor='black')
plt.xlabel("Énergie d'impact (J)")
plt.ylabel("Densité de probabilité")
plt.title("Spectre d'énergie d'impact des débris de glace")
plt.grid()
plt.show()


def find_impact_points(trajectories, x_impact=0.5):
    """
    Trouve les points d'impact sur le plan x = x_impact.

    Paramètres :
    - trajectories : liste de tableaux numpy (n, 3), les positions des trajectoires.
    - x_impact : valeur de x pour le plan d'impact.

    Retourne :
    - impact_points : liste des points (y, z) pour les trajectoires qui atteignent le plan.
    """
    impact_points = []

    for traj in trajectories:
        for i in range(1, len(traj)):
            # Vérifier si la trajectoire traverse le plan x = x_impact
            if traj[i - 1, 0] < x_impact <= traj[i, 0]:
                # Interpolation linéaire pour trouver y et z sur le plan
                alpha = (x_impact - traj[i - 1, 0]) / (traj[i, 0] - traj[i - 1, 0])
                y_impact = traj[i - 1, 1] + alpha * (traj[i, 1] - traj[i - 1, 1])
                z_impact = traj[i - 1, 2] + alpha * (traj[i, 2] - traj[i - 1, 2])


                # Vérifier si l'impact est dans les limites de y et z
                if -1 <= y_impact <= 1 and -1 <= z_impact <= 1:
                    impact_points.append((x_impact, y_impact, z_impact))
                break

    return np.array(impact_points)

# Trouver les points d'impact
impact_points = find_impact_points(trajectories, x_impact=0.5)
im = []
for p in impact_points :
    x, y, z = p[0], p[1], p[2]
    exclusion_radius = 0.5 * np.sqrt(0.5)  # Rayon d'exclusion
    distance_to_center = np.sqrt(y**2 + z**2)
    if distance_to_center >= exclusion_radius :
        im.append((x,y,z))
im = np.array(im)
# Visualisation des points d'impact
fig.add_trace(go.Scatter3d(
    x=im[:, 0],  # x est constant et vaut 0.5
    y=im[:, 1],  # Coordonnées y
    z=im[:, 2],  # Coordonnées z
    mode='markers',
    marker=dict(size=2, color='red', opacity=0.8),
    name='Points d\'impact'
))

# Afficher la figure mise à jour
fig.show()

# Affichage de la carte de chaleur des impacts
plt.figure(figsize=(8, 6))
plt.hist2d(impact_points[:, 1], impact_points[:, 2], bins=30, cmap='hot', density=True)
plt.colorbar(label='Densité des impacts')
plt.xlabel("y")
plt.ylabel("z")
plt.title("Carte de chaleur des points d'impact")
plt.grid()
plt.show()


# Simulation de 1000 trajectoires
n_simulations1 = 1000
trajectories1 = []
points_impact_y = []  # Stockage des points d'impact sur le mur (x = 0.5)

# Paramètres de la simulation
T_f = 0.5  # Temps final (s)
dt = 1 / (2 ** 10)  # Pas de temps (s)

for _ in range(n_simulations1):
    # Conditions initiales aléatoires
    x0 = np.random.uniform(0, 0.1)
    theta = 0
    y0 = 0.5 * np.sqrt(x0) * np.cos(theta)
    z0 = 0.5 * np.sqrt(x0) * np.sin(theta)
    v0 = np.array([0, -z0 * omega, y0 * omega])

    # Génération des paramètres aléatoires
    Cd = max(np.random.normal(0.47, 0.05), 0)  # Cd doit être positif
    R = max(np.random.normal(0.005, 0.001), 0)  # R doit être positif

    # Résolution de l'équation différentielle
    v_vectors, t = RK4(f, v0, dt, T_f, R, Cd)
    positions = calculate_positions(v_vectors, t, (x0, 0, 0))

    # Stockage des trajectoires
    trajectories1.append(positions)

    # Détection des impacts sur le mur x = 0.5
    for i in range(len(positions) - 1):
        if positions[i, 0] < 0.5 <= positions[i + 1, 0]:
            # Interpolation linéaire pour obtenir y à x = 0.5
            x1, z1 = positions[i, 0], positions[i, 2]
            x2, z2 = positions[i + 1, 0], positions[i + 1, 2]
            z_impact = z1 + (0.5 - x1) * (z2 - z1) / (x2 - x1)
            points_impact_y.append(z_impact)
            break

# Création de la figure des trajectoires
plt.figure(figsize=(10, 6))

for traj in trajectories1:
    x = traj[:, 0]
    z = traj[:, 2]
    mask = (z >= 0.5 * np.sqrt(x))
    plt.plot(x[mask], z[mask], alpha=0.5)

x_vals = np.linspace(0, 0.5, 500)
xco = np.zeros(500) + 0.5
zco = np.linspace(-0.5 * np.sqrt(0.5), 0.5 * np.sqrt(0.5), 500)
z_vals1 = 0.5 * np.sqrt(x_vals)
z_vals2 = -0.5 * np.sqrt(x_vals)

# Remplir entre les courbes noires
plt.fill_between(x_vals, z_vals1, z_vals2, color='black', alpha=0.5)

# Courbes et limites
plt.plot(x_vals, z_vals1, color='black', label='Le cône', alpha = 0.5)
plt.plot(x_vals, z_vals2, color='black', alpha = 0.5)
plt.plot(xco, zco, color="black", alpha = 0.5)

plt.title("Trajectoires des débris")
plt.xlabel("x (m)")
plt.ylabel("y (m)")
plt.xlim(-0.5, 1)
plt.ylim(-4, 4)
plt.legend()
plt.grid()
plt.show()

# Création de l'histogramme des points d'impact avec axes inversés
plt.figure(figsize=(10, 6))
plt.hist(points_impact_y, bins=30, color='blue', alpha=0.7, edgecolor='black', orientation='horizontal')
plt.title("Histogramme des points d'impact sur le plan de l'hélice")
plt.ylabel("y (m)")  # L'axe Y porte les valeurs maintenant
plt.xlabel("Fréquence")  # L'axe X porte les fréquences
plt.grid()
plt.show()


impact_energies = []
R_values = [max(np.random.normal(0.005, 0.001), 0) for _ in range(n_simulations)]  # Rayons utilisés

for i, traj in enumerate(trajectories):
    for j in range(1, len(traj)):
        if traj[j-1, 0] < 0.5 <= traj[j, 0]:
            # Interpolation linéaire de la vitesse à x=0.5
            alpha = (0.5 - traj[j-1, 0]) / (traj[j, 0] - traj[j-1, 0])
            v_impact = v_vectors[j-1] + alpha * (v_vectors[j] - v_vectors[j-1])

            # Calcul de l'énergie cinétique
            energy = 0.5 * m(R_values[i]) * np.linalg.norm(v_impact)**2
            impact_energies.append(energy)
            break

# 2. Filtrage des points hors de la zone exclue (cône)
valid_points = []
valid_energies = []

for idx, point in enumerate(impact_points):
    y, z = point[1], point[2]
    distance_to_center = np.sqrt(y**2 + z**2)
    exclusion_radius = 0.5 * np.sqrt(0.5)  # Rayon du cône à x=0.5

    if distance_to_center >= exclusion_radius:
        valid_points.append([y, z])
        valid_energies.append(impact_energies[idx])

valid_points = np.array(valid_points)
valid_energies = np.array(valid_energies)

# 3. Création de la grille pour la KDE
grid_size = 200j  # Résolution de la grille
y_min, y_max = valid_points[:, 0].min(), valid_points[:, 0].max()
z_min, z_max = valid_points[:, 1].min(), valid_points[:, 1].max()

y_grid, z_grid = np.mgrid[y_min:y_max:grid_size,
                          z_min:z_max:grid_size]

# 4. Calcul de la densité d'énergie pondérée
positions = np.vstack([y_grid.ravel(), z_grid.ravel()])
kde = gaussian_kde(valid_points.T, weights=valid_energies)
density = kde(positions).reshape(y_grid.shape)

# 5. Visualisation avancée
plt.figure(figsize=(12, 10))

# Carte de chaleur principale
heatmap = plt.pcolormesh(y_grid, z_grid, density,
                        cmap='inferno',
                        shading='auto')

# Contours pour les isovaleurs
contours = plt.contour(y_grid, z_grid, density,
                      levels=10, colors='white',
                      linewidths=0.5, alpha=0.7)

# Points d'impact individuels (transparents)
plt.scatter(valid_points[:, 0], valid_points[:, 1],
           c='cyan', s=5, alpha=0.15,
           label='Impacts individuels')

# Ajout du cercle d'exclusion (cône)
theta = np.linspace(0, 2*np.pi, 100)
exclusion_radius = 0.5 * np.sqrt(0.5)
circle_y = exclusion_radius * np.cos(theta)
circle_z = exclusion_radius * np.sin(theta)
plt.plot(circle_y, circle_z, 'r--', linewidth=1.5,
        label='Limite du cône')

# Configuration finale
plt.colorbar(heatmap, label='Densité d\'énergie (J/m²)')
plt.xlabel('Position y (m)', fontsize=12)
plt.ylabel('Position z (m)', fontsize=12)
plt.title('Densité d\'énergie des impacts sur l\'hélice\n(Fréquence × Énergie)', fontsize=14)
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()

plt.show()
# === Analyse d'érosion des pales ===
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import LinearSegmentedColormap
from scipy.interpolate import interp1d
from scipy.ndimage import gaussian_filter

# Vérification préliminaire des données d'impact
print(f"Nombre de points d'impact trouvés: {len(impact_points)}")
print(f"Nombre d'énergies calculées: {len(impact_energies)}")

# === Filtrage des points d'impact en excluant la zone du cône ===
# Rayon d'exclusion (cône)
rayon_exclusion = 0.5 * np.sqrt(0.5)

# Initialisation des listes pour stocker les points et énergies filtrés
points_filtres = []
energies_filtrees = []

# Boucle sur tous les points d'impact
for i, point in enumerate(impact_points):
    if i < len(impact_energies):  # Protection contre les index out of range
        y, z = point[1], point[2]  # On extrait y et z
        distance_centre = np.sqrt(y**2 + z**2)

        # Vérification si le point est en dehors de la zone d'exclusion
        if distance_centre >= rayon_exclusion:
            points_filtres.append([y, z])
            energies_filtrees.append(impact_energies[i])

# Conversion en tableaux numpy
points_filtres = np.array(points_filtres)
energies_filtrees = np.array(energies_filtrees)

print(f"Nombre de points d'impact valides après filtrage: {len(points_filtres)}")

# === Analyse par couronnes concentriques ===
# Paramètres
rayon_pale = 1.5  # Rayon maximal de la pale en mètres
nb_couronnes = 30  # Nombre de couronnes concentriques pour l'analyse

# Définition des rayons des couronnes
rayon_min = rayon_exclusion  # Rayon intérieur (cône)
rayon_max = rayon_pale       # Rayon extérieur (bord de la pale)

rayons_couronnes = np.linspace(rayon_min, rayon_max, nb_couronnes + 1)
rayons_centres = (rayons_couronnes[:-1] + rayons_couronnes[1:]) / 2

# Initialisation des tableaux pour l'érosion
erosion_couronnes = np.zeros(nb_couronnes)
compte_impacts = np.zeros(nb_couronnes, dtype=int)

# Paramètres matériaux
σ_el = 5e6        # Limite d'élasticité (Pa)
E = 10e9          # Module de Young (Pa)
ν = 0.3           # Coefficient de Poisson
ρ_p = 917         # Densité des particules de glace (kg/m³)
C = 0.3           # Efficacité de coupe
K = 1.0           # Rapport des aires de contact
F_s = 4.0         # Facteur de forme des particules
ε_D = 1.53e5      # Énergie de déformation

# Calcul de l'érosion pour chaque impact
for i, (point, energy) in enumerate(zip(points_filtres, energies_filtrees)):
    y, z = point  # Coordonnées du point d'impact
    distance_centre = np.sqrt(y**2 + z**2)

    # Déterminer la couronne d'appartenance
    idx_couronne = np.searchsorted(rayons_couronnes, distance_centre) - 1

    # Vérification que l'index est valide
    if 0 <= idx_couronne < nb_couronnes:
        # Calcul des paramètres d'impact
        # Rayon moyen des particules de glace (en m)
        rayon_particule = 0.005

        # Masse de la particule
        m_particule = (4/3) * np.pi * rayon_particule**3 * ρ_p

        # Vitesse approximative basée sur l'énergie
        v = np.sqrt(2 * energy / m_particule)

        # Angle d'impact (supposé perpendiculaire à la pale)
        θ = np.arctan2(1, 0)  # 90 degrés

        # Composantes de vitesse
        v_n = v * np.cos(θ)  # Vitesse normale
        v_t = v * np.sin(θ)  # Vitesse tangentielle

        # Vitesse critique élastique
        term = (1 - ν**2) / E
        v_el = (np.pi**2 / (2 * np.sqrt(10 * ρ_p))) * σ_el**2.5 * term**2

        # Érosion par déformation plastique
        V_d = 0.0
        if v_n > v_el:
            V_d = (0.5 * m_particule * (v_n - v_el)**2) / ε_D

        # Érosion par coupage
        V_c = 0.0
        θ_c = np.arctan(K/2)
        if θ >= θ_c:
            V_c = C * m_particule * v_t**2 * np.cos(θ)**2 / (2 * σ_el)
        else:
            V_c = C * 2 * m_particule * v_t**2 * np.sin(θ) * (K * np.cos(θ) - np.sin(θ)) / (σ_el * K**2)

        # Érosion totale
        total_erosion = F_s * (V_d + V_c)

        # Ajout à la couronne correspondante
        erosion_couronnes[idx_couronne] += total_erosion
        compte_impacts[idx_couronne] += 1

# Normalisation par couronne (érosion moyenne par impact)
for i in range(nb_couronnes):
    if compte_impacts[i] > 0:
        erosion_couronnes[i] /= compte_impacts[i]

# Lissage de la courbe d'érosion
erosion_couronnes = gaussian_filter(erosion_couronnes, sigma=1.5)

# Ajustement pour l'érosion totale (intégration de la fréquence)
# Plus on s'éloigne du centre, plus la surface est grande, donc plus d'impacts potentiels
for i in range(nb_couronnes):
    r_ext = rayons_couronnes[i+1]
    r_int = rayons_couronnes[i]
    surface_couronne = np.pi * (r_ext**2 - r_int**2)
    erosion_couronnes[i] *= surface_couronne  # Pondération par la surface

# === Modélisation de la forme optimale de la pale ===
# Paramètres de dimensionnement
epaisseur_min = 0.005  # 5mm d'épaisseur minimale
epaisseur_max = 0.025  # 25mm d'épaisseur maximale

# Normalisation de l'érosion (entre 0 et 1)
erosion_max = np.max(erosion_couronnes)
if erosion_max > 0:  # Éviter la division par zéro
    erosion_norm = erosion_couronnes / erosion_max
else:
    erosion_norm = np.zeros_like(erosion_couronnes)

# Fonction de correspondance érosion -> épaisseur
# Utilisation d'une fonction non-linéaire pour accentuer les zones critiques
epaisseurs = epaisseur_min + (epaisseur_max - epaisseur_min) * (1 - np.exp(-2 * erosion_norm))

# Interpolation pour obtenir une courbe lisse
interp_spline = interp1d(rayons_centres, epaisseurs, kind='cubic', fill_value="extrapolate")

# Génération de points plus fins pour l'affichage
rayons_interp = np.linspace(min(rayons_centres), max(rayons_centres), 200)
epaisseurs_interp = interp_spline(rayons_interp)

# === Visualisations ===
# Création d'une figure avec 3 sous-graphiques
plt.figure(figsize=(18, 12))
gs = gridspec.GridSpec(2, 2, width_ratios=[1, 1], height_ratios=[1, 1])

# 1. Carte de chaleur circulaire des couronnes concentriques
ax1 = plt.subplot(gs[0, 0], polar=True)

# Création des segments pour la visualisation
rayon_min = rayons_centres[0] - (rayons_centres[1] - rayons_centres[0])/2
rayon_max = rayons_centres[-1] + (rayons_centres[1] - rayons_centres[0])/2
rayons_limites = np.linspace(rayon_min, rayon_max, len(rayons_centres)+1)

# Normalisation pour la coloration
if np.max(erosion_couronnes) > 0:
    norm_erosion = erosion_couronnes / np.max(erosion_couronnes)
else:
    norm_erosion = np.zeros_like(erosion_couronnes)

# Création d'une palette de couleur personnalisée (bleu -> rouge)
cmap = LinearSegmentedColormap.from_list("erosion_cmap", ["blue", "green", "yellow", "red"])

# Tracé des anneaux concentriques
theta = np.linspace(0, 2*np.pi, 100)

for i in range(len(rayons_centres)):
    # Couleur basée sur l'érosion relative
    color = cmap(norm_erosion[i])

    # Rayon intérieur et extérieur de l'anneau
    r_in = rayons_limites[i]
    r_out = rayons_limites[i+1]

    # Création des coordonnées pour le remplissage
    r_coords = np.concatenate([np.ones_like(theta) * r_in, np.ones_like(theta) * r_out, [r_in]])
    theta_coords = np.concatenate([theta, theta[::-1], [theta[0]]])

    ax1.fill(theta_coords, r_coords, color=color, alpha=0.8)

# Configuration du graphique polaire
ax1.set_title("Carte de chaleur de l'érosion par couronnes", fontsize=14)
ax1.grid(True, linestyle='--', alpha=0.7)

# Barre de couleur
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(0, max(erosion_couronnes)))
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax1, pad=0.1)
cbar.set_label("Érosion cumulative (m³)", fontsize=12)

# 2. Profil d'érosion en fonction du rayon
ax2 = plt.subplot(gs[0, 1])
ax2.plot(rayons_centres, erosion_couronnes, 'b-', linewidth=2.5)
ax2.scatter(rayons_centres, erosion_couronnes, color='red', s=50)
ax2.set_xlabel("Distance radiale (m)", fontsize=12)
ax2.set_ylabel("Érosion cumulative (m³)", fontsize=12)
ax2.set_title("Profil d'érosion en fonction du rayon", fontsize=14)
ax2.grid(True, linestyle='--', alpha=0.7)
ax2.set_xlim(min(rayons_centres), max(rayons_centres))

# 3. Profil d'épaisseur optimale de la pale
ax3 = plt.subplot(gs[1, :])
ax3.plot(rayons_interp, epaisseurs_interp, 'g-', linewidth=3)

# Trouver les points de calcul correspondants aux rayons_centres
indices_points = []
for r in rayons_centres:
    idx = np.argmin(np.abs(rayons_interp - r))
    indices_points.append(idx)

ax3.scatter(rayons_centres, epaisseurs_interp[indices_points],
           color='blue', s=50, label="Points de calcul")
ax3.fill_between(rayons_interp, 0, epaisseurs_interp, color='lightgreen', alpha=0.3)
ax3.set_xlabel("Distance radiale (m)", fontsize=12)
ax3.set_ylabel("Épaisseur de la pale (m)", fontsize=12)
ax3.set_title("Profil optimal d'épaisseur de la pale", fontsize=14)
ax3.grid(True, linestyle='--', alpha=0.7)
ax3.legend()
ax3.set_xlim(min(rayons_interp), max(rayons_interp))

plt.tight_layout()
plt.show()



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import PathPatch
from matplotlib.path import Path
import matplotlib.patches as patches

def create_blade_profile():
    # Paramètres du profil
    L = 1.0  # Corde de référence
    f = 0.12 * L  # Cambrure maximale (12% de la corde)
    D = 0.4 * L  # Position de la cambrure maximale (40% de la corde)
    t = 0.15 * L  # Épaisseur maximale (15% de la corde)
    theta = np.radians(5)  # Angle de calage en radians

    # Fonction pour générer la ligne de cambrure
    def camber_line(x):
        if x <= D:
            return (f / D**2) * (2 * D * x - x**2)
        else:
            return (f / (1 - D)**2) * (1 - 2 * D + 2 * D * x - x**2)

    # Fonction pour la dérivée de la ligne de cambrure
    def camber_line_derivative(x):
        if x <= D:
            return (f / D**2) * (2 * D - 2 * x)
        else:
            return (f / (1 - D)**2) * (2 * D - 2 * x)

    # Fonction pour calculer l'épaisseur du profil à une position x
    def thickness_distribution(x):
        # Distribution d'épaisseur NACA classique
        return 5 * t * (0.2969 * np.sqrt(x/L) - 0.1260 * (x/L) - 0.3516 * (x/L)**2 + 0.2843 * (x/L)**3 - 0.1015 * (x/L)**4)

    # Création des points du profil
    def generate_airfoil_points(num_points=100):
        # Répartition des points le long de la corde
        x = np.linspace(0, L, num_points)

        # Calcul de la ligne de cambrure
        y_camber = np.array([camber_line(xi) for xi in x])

        # Calcul des angles de la ligne de cambrure
        dydx = np.array([camber_line_derivative(xi) for xi in x])
        phi = np.arctan(dydx)

        # Calcul de l'épaisseur à chaque point
        yt = np.array([thickness_distribution(xi) for xi in x])

        # Calcul des coordonnées des surfaces supérieure et inférieure
        x_upper = x - yt * np.sin(phi)
        y_upper = y_camber + yt * np.cos(phi)

        x_lower = x + yt * np.sin(phi)
        y_lower = y_camber - yt * np.cos(phi)

        # Retourner les points dans le bon ordre (du bord de fuite au bord d'attaque puis retour)
        return (
            np.concatenate((x_upper[::-1], x_lower[1:])),
            np.concatenate((y_upper[::-1], y_lower[1:]))
        )

    # Génération des points du profil
    x_profile, y_profile = generate_airfoil_points()

    # Création des points de la ligne de cambrure
    x_camber = np.linspace(0, L, 50)
    y_camber = np.array([camber_line(x) for x in x_camber])

    return x_profile, y_profile, x_camber, y_camber, L, D, f

def plot_blade_profile():
    # Obtention des données du profil
    x_profile, y_profile, x_camber, y_camber, L, D, f = create_blade_profile()

    # Création de la figure
    fig, ax = plt.subplots(figsize=(10, 6))

    # Tracer le profil
    ax.plot(x_profile, y_profile, 'k-', linewidth=2)

    # Tracer la ligne de cambrure
    ax.plot(x_camber, y_camber, 'magenta', linestyle='--', label='Ligne de cambrure')

    # Tracer la corde
    ax.plot([0, L], [0, 0], 'k--', alpha=0.7)

    # Marquer la position de cambrure maximale
    ax.plot([D], [camber_line(D)], 'ro')
    ax.vlines(D, 0, camber_line(D), colors='brown', linestyles='-')


    # Marquer les bords d'attaque et de fuite
    ax.plot(0, 0, 'ro', markersize=8)
    ax.plot(L, 0, 'ro', markersize=8)


    # Ajout des flèches pour l'angle de calage
    ax.annotate('', xy=(-0.1, 0), xytext=(-0.1, -0.1),
                arrowprops=dict(arrowstyle='<-', color='brown', lw=1.5))

    # Configuration des axes
    ax.set_xlim(-0.2, L+0.2)
    ax.set_ylim(-0.3, 0.3)
    ax.set_aspect('equal')
    ax.set_xlabel('x', fontsize=12)
    ax.set_ylabel('y', fontsize=12)
    ax.set_title('Profil de pale de turboréacteur', fontsize=14)
    ax.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.savefig('profil_pale_turboreacteur.png', dpi=300, bbox_inches='tight')
    plt.show()

    return fig, ax

def plot_thickness_distribution():
    # Obtention des données du profil
    x_profile, y_profile, x_camber, y_camber, L, D, f = create_blade_profile()

    # Calcul de l'épaisseur en fonction de la position x
    x_points = np.linspace(0, L, 100)
    thickness = np.array([thickness_distribution(x) for x in x_points])

    # Création de la figure
    fig, ax = plt.subplots(figsize=(10, 6))

    # Tracer la distribution d'épaisseur
    ax.plot(x_points/L, thickness/L, 'b-', linewidth=2)

    # Configuration des axes
    ax.set_xlim(0, 1)
    ax.set_xlabel('Distance radiale (m)', fontsize=12)
    ax.set_ylabel('Épaisseur (m)', fontsize=12)
    ax.set_title('Distribution d\'épaisseur le long de la pale', fontsize=14)
    ax.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.savefig('epaisseur_pale_turboreacteur.png', dpi=300)
    plt.show()

    return fig, ax

# Fonction pour calculer l'épaisseur
def thickness_distribution(x):
    L = 1.0  # Corde de référence
    t = 0.15 * L  # Épaisseur maximale (15% de la corde)
    return 5 * t * (0.2969 * np.sqrt(x/L) - 0.1260 * (x/L) - 0.3516 * (x/L)**2 + 0.2843 * (x/L)**3 - 0.1015 * (x/L)**4)

# Fonction pour la ligne de cambrure (déclarée hors des fonctions pour être accessible)
def camber_line(x):
    L = 1.0
    f = 0.12 * L
    D = 0.4 * L
    if x <= D:
        return (f / D**2) * (2 * D * x - x**2)
    else:
        return (f / (1 - D)**2) * (1 - 2 * D + 2 * D * x - x**2)

# Exécution des fonctions principales
if __name__ == "__main__":
    # Tracer le profil de la pale
    plot_blade_profile()

    # Tracer la distribution d'épaisseur
    plot_thickness_distribution()